In [1]:
import os
os.environ["JAX_PLATFORM_NAME"] = "cpu"
import netket as nk
import numpy as np
import matplotlib.pyplot as plt
import netket.nn as nknn
import flax.linen as nn
import jax.numpy as jnp

class FFNN(nn.Module):
    @nn.compact
    def __call__(self, x):
        x = nn.Dense(features=2*x.shape[-1], 
                     use_bias=True, 
                     param_dtype=np.complex128, 
                     kernel_init=nn.initializers.normal(stddev=0.01), 
                     bias_init=nn.initializers.normal(stddev=0.01)
                    )(x)
        x = nknn.log_cosh(x)
        x = jnp.sum(x, axis=-1)
        return x
    

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
def info(e):
    head   = list(e.parameters.keys())[0]
    body   = list(e.parameters[head].keys())
    bias   = e.parameters[head][body[0]]
    kernel = e.parameters[head][body[1]]
    return  head, body, list(bias), list(kernel)
def real(c):
    return float(np.real(c))  
def img(c):
    return float(np.imag(c))    
def r_i(c):
    return real(c),img(c)   

In [4]:
def run(it):
    print("------------",it,"-------------")
    J = [1, 0.2];L = 8;edge_colors = []
    for i in range(L):
        edge_colors.append([i, (i+1)%L, 1])
        edge_colors.append([i, (i+2)%L, 2])
    g = nk.graph.Graph(edges=edge_colors)
    sigmaz = [[1, 0], [0, -1]];  mszsz = (np.kron(sigmaz, sigmaz))
    exchange = np.asarray([[0, 0, 0, 0], [0, 0, 2, 0], [0, 2, 0, 0], [0, 0, 0, 0]])
    bond_operator = [
        (J[0] * mszsz).tolist(),    (J[1] * mszsz).tolist(),
        (-J[0] * exchange).tolist(),(J[1] * exchange).tolist(),
        ]
    bond_color = [1, 2, 1, 2];   sigmaz = [[1, 0], [0, -1]]
    mszsz = (np.kron(sigmaz, sigmaz));
    exchange = np.asarray([[0, 0, 0, 0], [0, 0, 2, 0], [0, 2, 0, 0], [0, 0, 0, 0]])
    bond_operator = [
        (J[0] * mszsz).tolist(),    (J[1] * mszsz).tolist(),
        (-J[0] * exchange).tolist(),(J[1] * exchange).tolist(),
    ]
    bond_color = [1, 2, 1, 2]
    hi = nk.hilbert.Spin(s=0.5, total_sz=0.0, N=g.n_nodes)
    op = nk.operator.GraphOperator(hi, graph=g, bond_ops=bond_operator, bond_ops_colors=bond_color)
    
    model = FFNN()    
    
    sa = nk.sampler.MetropolisExchange(hilbert=hi, graph=g, d_max = 2)
    vs = nk.vqs.MCState(sa, model, n_samples=1008)
    vs_i = vs;
    opt = nk.optimizer.Sgd(learning_rate=0.01)
    sr = nk.optimizer.SR(diag_shift=0.01)
    gs = nk.VMC(hamiltonian=op, optimizer=opt, variational_state=vs, preconditioner=sr)
    sf = []
    sites = []
    structure_factor = nk.operator.LocalOperator(hi, dtype=complex)
    for i in range(0, L):
        for j in range(0, L):
            structure_factor += (nk.operator.spin.sigmaz(hi, i)*nk.operator.spin.sigmaz(hi, j))*((-1)**(i-j))/L
    gs.run(out='test', n_iter=it, obs={'Structure Factor': structure_factor}) 
    print("------------  -------------")
    vs_f = vs
    
    return vs_i, vs_f, gs

In [5]:
it = 0; bia_i_list  = [];bia_f_list = [] ; kernel_i_list = []; kernel_f_list = []
#for i in range(40,50):
for i in range(0,10):
    #it = i*(10) + 10
    it = i + 10
    vs_i, vs_f, g  = run(it)
    head_i, body_i, bias_i, kernel_i   = info(vs_i); 
    head_f, body_f, bias_f, kernel_f  = info(vs_f)
    bia_i_list.append(bias_i)
    bia_f_list.append(bias_f)
    kernel_i_list.append(kernel_i)
    kernel_f_list.append(kernel_f)    
    break;

------------ 10 -------------


100%|█| 10/10 [00:00<00:00, 19.40it/s, Energy=-11.95-0.00j ± 0.10 [σ²=10.96, R̂=


------------  -------------


In [10]:
g.state

MCState(
  hilbert = Spin(s=1/2, total_sz=0.0, N=8),
  sampler = MetropolisSampler(rule = ExchangeRule(# of clusters: 28), n_chains = 16, n_sweeps = 8, reset_chains = False, machine_power = 2, dtype = <class 'float'>),
  n_samples = 1008,
  n_discard_per_chain = 100,
  sampler_state = MetropolisSamplerState(# accepted = 16635/20864 (79.73063650306749%), rng state=[ 234656659 1159882200]),
  n_parameters = 144)

In [9]:
g

Vmc(
  step_count = 10,
  state = MCState(hilbert = Spin(s=1/2, total_sz=0.0, N=8), sampler = MetropolisSampler(rule = ExchangeRule(# of clusters: 28), n_chains = 16, n_sweeps = 8, reset_chains = False, machine_power = 2, dtype = <class 'float'>), n_samples = 1008))

In [6]:
vs_f

MCState(
  hilbert = Spin(s=1/2, total_sz=0.0, N=8),
  sampler = MetropolisSampler(rule = ExchangeRule(# of clusters: 28), n_chains = 16, n_sweeps = 8, reset_chains = False, machine_power = 2, dtype = <class 'float'>),
  n_samples = 1008,
  n_discard_per_chain = 100,
  sampler_state = MetropolisSamplerState(# accepted = 16635/20864 (79.73063650306749%), rng state=[ 234656659 1159882200]),
  n_parameters = 144)

In [11]:
vs_f

MCState(
  hilbert = Spin(s=1/2, total_sz=0.0, N=8),
  sampler = MetropolisSampler(rule = ExchangeRule(# of clusters: 28), n_chains = 16, n_sweeps = 8, reset_chains = False, machine_power = 2, dtype = <class 'float'>),
  n_samples = 1008,
  n_discard_per_chain = 100,
  sampler_state = MetropolisSamplerState(# accepted = 16635/20864 (79.73063650306749%), rng state=[ 234656659 1159882200]),
  n_parameters = 144)

In [7]:
vs_f.parameters

{'Dense_0': {'bias': Array([-0.00270085-0.01020368j, -0.0063255 +0.01663j   ,
         -0.00566629-0.00173295j,  0.01165582-0.00322587j,
         -0.00109178+0.00325852j,  0.00692918-0.00133645j,
          0.00536029-0.00450292j,  0.0097808 +0.01201983j,
         -0.00858378+0.00624034j, -0.0056656 +0.00561149j,
         -0.0070984 -0.00424987j, -0.00160859+0.00041818j,
         -0.00346794+0.00842797j, -0.00477604+0.01029099j,
         -0.00582455+0.00022033j,  0.00714422-0.00463331j],      dtype=complex128),
  'kernel': Array([[ 0.12942524-0.05155985j,  0.03200601+0.04617393j,
          -0.01360799-0.02912333j, -0.04020112-0.10784593j,
          -0.08405289-0.02171291j,  0.03339837+0.02709561j,
          -0.12237747-0.0441213j ,  0.08344419-0.04289496j,
          -0.07549742-0.0455817j , -0.07444407-0.08477516j,
          -0.02928698+0.04822322j,  0.04404021-0.16529122j,
          -0.01930931-0.04113447j,  0.01496   -0.07737549j,
           0.02595941-0.11317653j,  0.04146922-0.01141

In [ ]:
g

In [ ]:
g.state

In [ ]:
g.state.parameters.keys()

In [ ]:
g.state.parameters['Dense_0']

In [ ]:
vs_i.parameters['Dense_0']

In [ ]:
g.state.parameters['Dense_0']

In [ ]:
vs.parameters[head]

In [ ]:
g

In [ ]:
vs_i

In [ ]:
g.state

In [ ]:
head_g, body_g, bias_g, kernel_g   = info(g.state)
head_i, body_i, bias_i, kernel_i   = info(vs_i)

In [ ]:
for l in kernel_i:
        for m in l:
            print(r_i(m)) 
            break;

In [ ]:
for l in kernel_g:
        for m in l:
            print(r_i(m)) 
            break;

In [ ]:
for b in bias_g:
    print(r_i(b))
    break;

In [ ]:
for b in bias_i:
    print(r_i(b))
    break;

In [ ]:
for b in bias_i:
    print(r_i(b))
    break;

In [ ]:
g

In [ ]:
vs_i

In [ ]:
for bia in bia_i_list:
    for b in bia:
        print(r_i(b))

In [ ]:
for bia in bia_f_list:
    for b in bia:
        print(r_i(b))

In [ ]:
for kernel in kernel_i_list:
    for l in kernel:
        for m in l:
            print(r_i(m))    

In [ ]:
for kernel in kernel_f_list:
    for l in kernel:
        for m in l:
            print(r_i(m))      